In [61]:
pip install jupyterthemes

Note: you may need to restart the kernel to use updated packages.Collecting jupyterthemes



In [112]:
!jt -t monokai

In [1]:
#Run in Anaconda Prompt
#conda activate Tesis
#python -m ipykernel install --user --name=<myenv> 
conda env list

# conda environments:
#
base                  *  C:\Users\carlos.perez-sintica\anaconda3
Tesis                    C:\Users\carlos.perez-sintica\anaconda3\envs\Tesis


Note: you may need to restart the kernel to use updated packages.


In [3]:
#!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
#!jupyter nbextension install --py pythreejs
!jupyter nbextension enable --py pythreejs
#!jupyter labextension list

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-threejs/extension...
      - Validating: ok


In [2]:
from datetime import datetime

In [3]:
start = datetime.now()

import igl
import numpy as np
import math
import random as rnd
import scipy as sp
from scipy.spatial import KDTree
import meshplot as mp
from meshplot import plot, subplot, interact

end = datetime.now()
print(end - start)

0:00:03.983560


In [4]:
start = datetime.now()
v, f = igl.read_triangle_mesh("data/30_original.off") #("3dPotteryDs/3D Models/Lagynos/qpvase5.obj")
end = datetime.now()
print(end - start)

0:00:03.340951


In [6]:
#from IPython.display import display, HTML
#display(HTML(mp.plot(v, f, v[:, 0]).to_html()))

#vw = mp.plot(v, f, v[:, 0])
#vw.save("30_original.html")

In [5]:
start = datetime.now()

n_vertex = igl.per_vertex_normals(v, f)
k = igl.gaussian_curvature(v, f)
kn = k[(~np.isnan(k)) & (k < 2*math.pi)]
#v1, v2, k1, k2 = igl.principal_curvature(v, f)
#v1_3, v2_3, k1_3, k2_3 = igl.principal_curvature(v, f, 3, True)

#Returns list of lists containing at index i the adjacent vertices of vertex i
adj_lst = igl.adjacency_list(f) 

#Returns
#VF (size: 3*#F) List of faces indice on each vertex, so that VF(NI(i)+j) = f, means that face f is the jth face 
# (in no particular order) incident on vertex i.
#VFi (size: #V+1) cumulative sum of vertex-triangle degrees with a preceeding zero. 
# “How many faces” have been seen before visiting this vertex and its incident faces.
VF, VFi = igl.vertex_triangle_adjacency(f, len(v))
bv = igl.is_border_vertex(v, f)

end = datetime.now()
print(end - start)

0:00:01.530063


In [8]:
len(k), len(kn)

(508963, 436707)

In [34]:
print(round(np.percentile(abs(kn), 95), 8), round(np.percentile(abs(kn), 85), 8), round(np.percentile(abs(kn), 75), 8), \
      round(np.percentile(abs(kn), 65), 8), round(np.percentile(abs(kn), 55), 8), round(np.percentile(abs(kn), 45), 8), \
      round(np.percentile(abs(kn), 35), 8), round(np.percentile(abs(kn), 25), 8), round(np.percentile(abs(kn), 15), 8), \
      round(np.percentile(abs(kn), 5), 8), round(np.percentile(abs(kn), 33), 8))

0.00410996 0.00081111 0.00037342 0.000228 0.00015097 0.00010226 6.781e-05 4.191e-05 2.192e-05 6.38e-06 6.204e-05


In [6]:
#Calculate curvatures for sampled points
area_t = 0.5 * sum(igl.doublearea(v, f))
v_size = 3 * (v.shape[0]**(1/2)) #min(round(area_t/4), round(v.shape[0] / 100))
v_ratio = math.floor(v.shape[0] / v_size)
v_size = math.floor(v.shape[0] / v_ratio)
round(area_t, 4), v.shape[0], v_ratio, v_size, v_ratio * v_size, len(bv)

(40107.8031, 508963, 237, 2147, 508839, 508962)

In [7]:
VF[0:20], VFi[0:7], len(f) * 3, len(VF), len(VFi), len(v)

(array([  4610,   4628,  52905, 797446, 845417, 845418,   4627,   4628,
          4630,  52905,  52912,  52914,   4630,   4631,  12158,  52914,
         66280,  12157,  12158,  12181], dtype=int32),
 array([ 0,  0,  6, 12, 17, 24, 30], dtype=int32),
 2600082,
 2600082,
 508964,
 508963)

In [11]:
#v_indices[len(v_indices) - 5:len(v_indices) - 1]
ivx = 3
VF[VFi[ivx]:VFi[ivx+1]]

array([ 4630,  4631, 12158, 52914, 66280], dtype=int32)

In [7]:
def calculate_principal(ivx): #curvature with 3-rounds
    lfaces, lvertices = [], []
    lvertices.append(ivx)
    lvx_r1, lvx_r2 = [], []

    for fx in VF[VFi[ivx]:VFi[ivx+1]]:
        lfaces.append(fx) #1-round faces

    for vr1 in adj_lst[ivx]:
        if vr1 not in lvertices:
            lvertices.append(vr1) #1-round vertices
            lvx_r1.append(vr1)
            
    for vr1 in lvx_r1:
        for fx2 in VF[VFi[vr1]:VFi[vr1+1]]:
            if fx2 not in lfaces:
                lfaces.append(fx2)
                                
        for vr2 in adj_lst[vr1]:
            if vr2 not in lvertices:
                lvertices.append(vr2)
                lvx_r2.append(vr2)
    
    for vr2 in lvx_r2:
        for fx3 in VF[VFi[vr2]:VFi[vr2+1]]:
            if fx3 not in lfaces:
                lfaces.append(fx3)

        for vr3 in adj_lst[vr2]:
            if vr3 not in lvertices:
                lvertices.append(vr3)

    dtv = {} #vertex dictionary
    for ix, vertex in enumerate(lvertices):
        dtv[vertex] = ix    
    
    vc = np.array([v[lvertices[i]] for i in range(len(lvertices))])
    fc = np.array([[dtv[f[lfaces[i]][0]], dtv[f[lfaces[i]][1]], dtv[f[lfaces[i]][2]]] for i in range(len(lfaces))], dtype=int)
    v1c, v2c, k1c, k2c = igl.principal_curvature(vc, fc)
    return v1c[0], v2c[0], k1c[0], k2c[0]

In [8]:
for ix in adj_lst[1]:
    v1c, v2c, k1c, k2c = calculate_principal(ix)
    print(ix, np.cross(v1c, v2c), k1c, k2c, n_vertex[ix])

2 [-0.7208037  -0.09002938  0.68726759] -0.012231431290679138 0.03915901996991442 [-0.72335591 -0.08842486  0.68478994]
2717 [ 0.72333969  0.07561711 -0.68633938] -0.0221066244699853 0.028532910812374544 [-0.72464731 -0.07320363  0.68522077]
2758 [ 0.7213237   0.07863276 -0.68811991] -0.018898862119275814 0.03771974162346207 [-0.72323175 -0.07933341  0.68603356]
31696 [ 0.72383696  0.09138069 -0.68389299] -0.008034118942899808 0.054678282164025635 [-0.72538058 -0.09122433  0.68227643]
408030 [-0.72645148 -0.08196634  0.68231208] -0.022570384659111726 0.016212418881501354 [-0.72719531 -0.08096559  0.68163887]
448939 [ 0.72801182  0.08945377 -0.67970348] -0.014943681067951579 0.024483940035763187 [-0.72862387 -0.08972695  0.67901129]


In [8]:
start = datetime.now()
v_indices = [] #np.array([0 for i in range(v_size)])
v1_p, v2_p, k1_p, k2_p = [], [], [], []
k_p = np.percentile(abs(kn), 33)

from datetime import datetime
start = datetime.now()

for i in range(0, v_size):
    while True:
        index = rnd.randint(v_ratio * i, min(v_ratio * (i + 1) - 1, v.shape[0] - 1))
        if len(adj_lst[index]) > 0:
            break            
    
    adj_neighbors = False
    
    if not bv[index]:
        adj_neighbors = True
        
        for nb in adj_lst[index]:
            if len(adj_lst[nb]) == 0:
                adj_neighbors = False
        
    if adj_neighbors and abs(k[index]) > k_p:
        v_indices.append(index)        
        v1p, v2p, k1p, k2p = calculate_principal(index)
        v1_p.append(v1p)
        v2_p.append(v2p)
        k1_p.append(k1p * np.sign(k2p))
        k2_p.append(k2p * np.sign(k2p))

end = datetime.now()
print(end - start)

0:00:02.790295


In [18]:
len(k1_p)

1379

In [152]:
ax = [0, 0, 0, 0]
for xi in range(0, len(k1_p)):
    if k1_p[xi] > 0:
        if k2_p[xi] > 0:
            ax[0] += 1
        else:
            ax[3] += 1
    else:
        if k2_p[xi] > 0:
            ax[1] += 1
        else:
            ax[2] += 1
            #print(k1_p[xi], k2_p[xi])

print(ax)

[350, 871, 176, 0]


In [40]:
np.percentile(np.abs(k1_p), 50), np.percentile(np.abs(k2_p), 50)

(0.026659197142526504, 0.039173466359112064)

In [179]:
np.percentile(np.abs(k1_p), 66), np.percentile(np.abs(k2_p), 66)

(0.031080422272298687, 0.043022749878669046)

In [9]:
oVertex = []
v1_c, v2_c, k1_c, k2_c = [], [], [], []

for j in range(0, len(v_indices)):
    if (abs(k1_p[j]) > np.percentile(np.abs(k1_p), 50)) or (abs(k2_p[j]) > np.percentile(np.abs(k2_p), 50)):
        #print(v_indices[j], k1_c[j], k2_c[j], k_c[v_indices[j]])
        oVertex.append(v_indices[j])
        v1_c.append(v1_p[j])
        v2_c.append(v2_p[j])
        k1_c.append(k1_p[j])
        k2_c.append(k2_p[j])

len(v_indices), len(oVertex)

(1385, 1074)

In [151]:
ax = [0, 0, 0, 0]
for xi in range(0, len(k1_c)):
    if k1_c[xi] > 0:
        if k2_c[xi] > 0:
            ax[0] += 1
        else:
            ax[3] += 1
    else:
        if k2_c[xi] > 0:
            ax[1] += 1
        else:
            ax[2] += 1
            #print(k1_p[xi], k2_p[xi])

print(ax)

[222, 443, 117, 0]


In [43]:
print(round(np.percentile(k1_p, 95), 8), round(np.percentile(k1_p, 75), 8), round(np.percentile(k1_p, 55), 8), \
      round(np.percentile(k1_p, 35), 8), round(np.percentile(k1_p, 15), 8), round(np.percentile(k1_p, 5), 8))
print(round(np.percentile(k2_p, 95), 8), round(np.percentile(k2_p, 75), 8), round(np.percentile(k2_p, 55), 8), \
      round(np.percentile(k2_p, 35), 8), round(np.percentile(k2_p, 15), 8), round(np.percentile(k2_p, 5), 8))

0.11226141 0.02304306 -0.00310667 -0.02068575 -0.04494122 -0.11304049
0.42315511 0.08615237 0.04392724 0.02546117 0.01075132 0.0029079


In [42]:
print(round(np.percentile(k1_c, 95), 8), round(np.percentile(k1_c, 75), 8), round(np.percentile(k1_c, 55), 8), \
      round(np.percentile(k1_c, 35), 8), round(np.percentile(k1_c, 15), 8), round(np.percentile(k1_c, 5), 8))
print(round(np.percentile(k2_c, 95), 8), round(np.percentile(k2_c, 75), 8), round(np.percentile(k2_c, 55), 8), \
      round(np.percentile(k2_c, 35), 8), round(np.percentile(k2_c, 15), 8), round(np.percentile(k2_c, 5), 8))

0.15221468 0.02997905 0.00142241 -0.02804728 -0.05403061 -0.14125293
0.4735715 0.13511358 0.05968604 0.03932734 0.01367592 0.00442542


In [14]:
TH_RATIO_K = 0.9
TH_COSINE = 0.75

#distance / bounding_box, angle
CLUSTER_EP = [0.125, 0.125] #0.125

In [15]:
length_x = np.percentile(v[1:,0], 99.9) - np.percentile(v[1:,0], 0.1)
length_y = np.percentile(v[1:,1], 99.9) - np.percentile(v[1:,1], 0.1)
length_z = np.percentile(v[1:,2], 99.9) - np.percentile(v[1:,2], 0.1)

diagonal_bbox = math.sqrt(length_x * length_x + length_y * length_y + length_z * length_z)
diagonal_bbox, length_x, length_y, length_z

(221.52662417560558, 142.7342, 103.81300000000002, 133.88)

In [137]:
np.percentile(v[1:,0], 99.5), np.percentile(v[1:,0], 0.5), np.percentile(v[1:,1], 99.5), np.percentile(v[1:,1], 0.5), np.percentile(v[1:,2], 99.5), np.percentile(v[1:,2], 0.5)

(61.2375, -76.184, -155.006, -257.009, 116.127, -12.861053499999999)

In [138]:
np.percentile(v[1:,0], 99.9), np.percentile(v[1:,0], 0.1), np.percentile(v[1:,1], 99.9), np.percentile(v[1:,1], 0.1), np.percentile(v[1:,2], 99.9), np.percentile(v[1:,2], 0.1)

(64.7793, -77.9549, -154.297, -258.11, 118.252, -15.628)

In [136]:
max(v[0:,0]), min(v[0:,0]), max(v[0:,1]), min(v[0:,1]), max(v[0:,2]), min(v[0:,2])

(66.1753, -78.5671, -0.0, -259.101, 118.817, -16.9141)

In [16]:
def projectPointPlane(point, plane, planeD):
    k = point.dot(plane) + planeD
    return point - k * plane

def distanceMetricRx(rxVectorA, rxProjA, rxVectorB, rxProjB):
    module = np.linalg.norm(rxVectorA) * np.linalg.norm(rxVectorB)
    angle = math.acos(min(1.0, abs(np.dot(rxVectorA, rxVectorB)) / module))
    
    origin = np.array([0, 0, 0])
    #projectionA = projectPointPlane(origin, rxPairA[2], rxPairA[3])
    #projectionB = projectPointPlane(origin, rxPairB[2], rxPairB[3])
    distance = np.linalg.norm(rxProjA - rxProjB)
    
    return distance / diagonal_bbox, angle

In [29]:
start = datetime.now()
lk1 = np.percentile(k1_c, 99.5) - np.percentile(k1_c, 0.5)
lk2 = np.percentile(k2_c, 99.5) - np.percentile(k2_c, 0.5)
rd = np.sqrt(lk1*lk1 + lk2*lk2) / 16
rd, lk1, lk2, len(k1_c)

kl = [[abs(k1_c[i]), abs(k2_c[i])] for i in range(len(k1_c))]
tk = KDTree(kl)

px = 542
idx = tk.query_ball_point([k1_c[px], k2_c[px]], r = rd, return_sorted=True)
idl = [i for i in idx if i > px]
len(idx), len(idl)

#for j in idl:
#    print(j)

end = datetime.now()
print(end - start)

0:00:00.007559


In [118]:
do1 = [3,4,5]
do2 = [3,0.5,6]
np.dot(do1, do2)

41.0

In [ ]:
#At elliptical points, both principal curvatures have the same sign, and the surface is locally convex.
#At hyperbolic points, the principal curvatures have opposite signs, and the surface will be locally saddle shaped.

In [17]:
# iPoints : indexes list of points Sampled (KdTree)
def baseQReflection(iPoints): 
    sample_size = len(iPoints)
    lxPairs = []
    
    lk1 = np.percentile(k1_c, 99.5) - np.percentile(k1_c, 0.5)
    lk2 = np.percentile(k2_c, 99.5) - np.percentile(k2_c, 0.5)
    rd = np.sqrt(lk1*lk1 + lk2*lk2) / 16
    kl = [[k1_c[i], k2_c[i]] for i in range(len(k1_c))]
    tk = KDTree(kl)

    for i in range(0, sample_size - 1):
        idx = tk.query_ball_point([k1_c[i], k2_c[i]], r = rd, return_sorted=True)
        idl = [kx for kx in idx if kx > i]
        
        for j in idl: #range(i + 1, sample_size):
            ratio_1, ratio_2 = k1_c[i] / k1_c[j], k2_c[i] / k2_c[j]
            ix, jx = iPoints[i], iPoints[j]
            
            if TH_RATIO_K < ratio_1 < 1/TH_RATIO_K and TH_RATIO_K < ratio_2 < 1/TH_RATIO_K: #ratio_1 * ratio_2 > 0
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D)
                    
                    #Use projection point instead of D-component
                    #xcPair = [ix, jx, reflexNormal.tolist(), projection.tolist(), i, j]
                    xcPair = [ix, jx, reflexNormal, projection, i, j]
                    lxPairs.append(xcPair)
    
    return lxPairs

In [134]:
# iPoints : indexes list of points Sampled
def baseReflection(iPoints): 
    sample_size = len(iPoints)
    lxPairs = []
    
    for i in range(0, sample_size - 1):        
        for j in range(i + 1, sample_size):
            ratio_1, ratio_2 = abs(k1_c[i] / k1_c[j]), abs(k2_c[i] / k2_c[j])
            
            ix, jx = iPoints[i], iPoints[j]
            
            if TH_RATIO_K < ratio_1 < 1/TH_RATIO_K and TH_RATIO_K < ratio_2 < 1/TH_RATIO_K:
                reflexNormal = v[ix] - v[jx]
                reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)
                    
                normalBalance = n_vertex[ix] + n_vertex[jx]
                normalBalance = normalBalance / np.linalg.norm(normalBalance)
                                
                if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
                    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
                        reflexNormal = -reflexNormal
                    
                    middlePoint = (v[ix] + v[jx]) / 2
                    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
                    origin = np.array([0, 0, 0])
                    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D)
                    
                    #Use projection point instead of D-component
                    xcPair = [ix, jx, reflexNormal, projection, i, j]
                    lxPairs.append(xcPair)
    
    return lxPairs

In [18]:
start = datetime.now()

lRfxPairs = baseQReflection(oVertex) #v_indices
print(len(lRfxPairs))

end = datetime.now()
print(end - start)

847
0:00:00.585162


In [69]:
start = datetime.now()

lRfxPairs = baseReflection(oVertex) #v_indices
len(lRfxPairs)

end = datetime.now()
print(end - start)

0:00:00.532666


In [187]:
type(lRfxPairs[0][3])

list

In [183]:
def clusterRfx(pairs):
    cluster_set = []
    cluster_center = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRx(pairs[i][2], pairs[i][3], cluster_center[c][0], cluster_center[c][1])
            
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:
                cluster_center[c][0] = (cluster_center[c][0] * len(cluster_set) + pairs[i][2]) / (len(cluster_set) + 1)
                cluster_center[c][1] = (cluster_center[c][1] * len(cluster_set) + pairs[i][3]) / (len(cluster_set) + 1)
                cluster_center[c][0] = cluster_center[c][0] / np.linalg.norm(cluster_center[c][0])
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [pairs[i], [], []]
            cluster_set.append(cluster)
            cluster_center.append(pairs[i][2:4])
            c = len(cluster_set) - 1
        
        cluster_set[c][1].append(i)
        cluster_set[c][2].append(pairs[i])
    
    return cluster_set, cluster_center

In [189]:
start = datetime.now()

s_cluster, c_cluster = clusterRfx(lRfxPairs)

end = datetime.now()
print(end - start)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [134]:
for x in range(0, len(s_cluster)):
    if len(s_cluster[x][1]) > 12:
        print(x, len(s_cluster[x][1]), c_cluster[x])

3 33 [array([ 0.99438395, -0.03327343, -0.10046613]), array([-7.55271902,  0.25272417,  0.76307797])]
14 28 [array([ 0.92054967, -0.05505866, -0.3867258 ]), array([-9.07531652,  0.54280042,  3.81256884])]
15 15 [array([ 0.94071783,  0.28505625, -0.18382842]), array([-75.12549572, -22.76452263,  14.68049248])]
18 20 [array([0.97595426, 0.21549125, 0.03281468]), array([-51.93851928, -11.46805428,  -1.74633813])]
21 20 [array([0.90560524, 0.2027628 , 0.37251361]), array([-31.76962356,  -7.1131409 , -13.0681855 ])]
23 14 [array([-0.18012687,  0.98034978, -0.0804277 ]), array([  40.69728567, -221.49708036,   18.17157715])]
30 24 [array([-0.57172194, -0.02144893,  0.82016704]), array([-27.14935554,  -1.01854519,  38.94726603])]
36 14 [array([ 0.0229093 , -0.08038989,  0.99650019]), array([ 1.55419028, -5.4537315 , 67.60358092])]
41 15 [array([0.6057859 , 0.04541085, 0.79433073]), array([14.84966454,  1.11315883, 19.47147483])]
43 14 [array([0.70695531, 0.01643355, 0.70706727]), array([18.612

In [19]:
def correctDirection(nVector):
    nVector = nVector * np.sign(nVector[np.argmax(abs(nVector))])
    return nVector

def normalized(X):
    return X / np.linalg.norm(X)

def projectPointLine(point, line, pointLine):
    ap = point - pointLine
    ab = line
    return pointLine + np.dot(ap, ab)/np.dot(ab, ab) * ab

def distancePointLine(point, line, pointLine):
    return np.linalg.norm(point - projectPointLine(point, line, pointLine))

def planesIntersection(pairM, pairN):
    U = normalized(np.cross(pairM[2], pairN[2]))
    M = np.array((pairM[2], pairN[2], U))
    X = np.array((np.dot(pairM[2], pairM[3]), np.dot(pairN[2], pairN[3]), 0.))
    
    origin = np.array([0, 0, 0])
    return correctDirection(U), projectPointLine(origin, U, np.linalg.solve(M, X))

def reflectPointLine(point, line, pointLine):
    projection = projectPointLine(point, line, pointLine)
    return 2 * projection - point

def reflectPointPlane(point, line, pointLine):
    projection = projectPointPlane(point, line, pointLine)
    return 2 * projection - point

In [117]:
type(lRfxPairs[0]), type(lRfxPairs[0]), lRfxPairs[0][2:4], type(lRfxPairs[0][2])

(list,
 list,
 [array([ 0.06808038, -0.63529001,  0.76926697]),
  array([  9.81896544, -91.62537492, 110.94834358])],
 numpy.ndarray)

In [65]:
v[1], v[2], v[1,1], v[0:,1], v[2252], v[170243]

(array([ -41.1933, -206.716 ,   84.6047]),
 array([ -41.1203, -206.716 ,   84.6832]),
 -206.716,
 array([  -0.   , -206.716, -206.716, ..., -160.777, -160.763, -160.673]),
 array([  63.7168, -159.61  ,   40.1134]),
 array([  39.608 , -254.176 ,   24.3943]))

In [ ]:
px = 542
idx = tk.query_ball_point([k1_c[px], k2_c[px]], r = rd, return_sorted=True)

idl = [i for i in idx if i > px]
len(idx), len(idl)

#for j in idl:
#    print(j)

In [49]:
a = [1, 2, 3, 4, 5]
b = [5, 2, 6, 3, 9]
result = [i for i in a if i in b]
#result = (set(a) & set(b))
print("The common elements are:", result)

The common elements are: [2, 3, 5]


In [105]:
#array = [[0.9, 0.6, 0.7], [0.9, 0.65, 0.8], [0.85, 0.55, 0.45], [0.7, 0.75, 0.85], [0.65, 0.5, 0.5], [0.87, 0.85, 0.9]]
array = np.array([[0.65, 0.6, 0.7], [0.9, 0.65, 0.8], [0.85, 0.55, 0.45], [0.7, 0.75, 0.85], [0.65, 0.5, 0.5], \
                  [0.87, 0.85, 0.9]])
kdt_p = KDTree(array)
len(array), len(array[0]), type(array)

(6, 3, numpy.ndarray)

In [108]:
idx = kdt_p.query_ball_point([0.86, 0.7, 0.65], r = 0.3, return_sorted=True)
type(idx)

list

In [138]:
type(np.array(lRfxPairs, dtype=object)[0:,2][0])

list

In [137]:
type(lRfxPairs[0][2]), lRfxPairs[0][2]

(list, [0.06808038332175165, -0.635290009188027, 0.7692669664249466])

In [14]:
def BaseContinuousRotation(pairs):
    lrPairs = []
    for m in range(0, len(pairs) - 1):
        for n in range(m+1, len(pairs)):
            rpM = pairs[m]
            rpN = pairs[n]
            ratio_k1 = (abs(k1_c[rpM[4]]) + abs(k1_c[rpM[5]])) / (abs(k1_c[rpN[4]]) + abs(k1_c[rpN[5]]))
            ratio_k2 = (abs(k2_c[rpM[4]]) + abs(k2_c[rpM[5]])) / (abs(k2_c[rpN[4]]) + abs(k2_c[rpN[5]]))
            not_parallel = abs(np.dot(rpM[2], rpN[2])) < TH_RATIO_K
            
            if TH_RATIO_K < ratio_k1 < 1 / TH_RATIO_K and TH_RATIO_K < ratio_k2 < 1 / TH_RATIO_K and not_parallel:
                #print(m, rpM[2], n, rpN[2])
                line = planesIntersection(rpM, rpN)
                
                p1_i = (v[rpM[0]] + v[rpM[1]]) / 2
                p2_i = (v[rpN[0]] + v[rpN[1]]) / 2
                distance_1i = distancePointLine(p1_i, line[0], line[1])
                projection_1i = projectPointLine(p1_i, line[0], line[1])
                
                distance_2i = distancePointLine(p2_i, line[0], line[1])
                projection_2i = projectPointLine(p2_i, line[0], line[1])
                
                if np.linalg.norm(projection_1i - projection_2i) < CLUSTER_EP[0] and distance_1i - distance_2i < CLUSTER_EP[0]:
                    #line[0], line[1]
                    rtPair = [rpM[0:2], rpN[0:2], line[0], line[1], rpM[4:6], rpN[4:6]]
                    lrPairs.append(rtPair)
    return lrPairs

In [198]:
lRfxPairs[0][5], k1_c[480]

(480, -0.05282461443192877)

In [62]:
avg_k = []
#Average curvatures
for i in range(0, len(lRfxPairs)):
    avg_k.append([(k1_c[lRfxPairs[i][4]] + k1_c[lRfxPairs[i][5]])/2, (k2_c[lRfxPairs[i][4]] + k2_c[lRfxPairs[i][5]])/2])

kt_rt = KDTree(avg_k)

In [34]:
len(avg_k), len(lRfxPairs)

(809, 809)

In [37]:
start = datetime.now()
lk1 = np.percentile(k1_c, 99.5) - np.percentile(k1_c, 0.5)
lk2 = np.percentile(k2_c, 99.5) - np.percentile(k2_c, 0.5)
rd = np.sqrt(lk1*lk1 + lk2*lk2) / 16
rd, lk1, lk2, len(k1_c)

kl = [[abs(k1_c[i]), abs(k2_c[i])] for i in range(len(k1_c))]
tk = KDTree(kl)

px = 542
idx = tk.query_ball_point([k1_c[px], k2_c[px]], r = rd, return_sorted=True)
idl = [i for i in idx if i > px]
len(idx), len(idl)

#for j in idl:
#    print(j)

end = datetime.now()
print(end - start)

0:00:00.010992


In [63]:
pair = lRfxPairs[3]
(k1_c[pair[4]] + k1_c[pair[5]])/2, (k2_c[pair[4]] + k2_c[pair[5]])/2

(-0.031825351568282964, 0.03619138519449931)

In [65]:
lk0 = [row[0] for row in avg_k]
lk1 = [row[1] for row in avg_k]
print(min(lk0), np.percentile(lk0, 0.5), np.percentile(lk0, 99.5), max(lk0))
print(min(lk1), np.percentile(lk1, 0.5), np.percentile(lk1, 99.5), max(lk1))

-0.21267643599298658 -0.13880676198768732 0.15492941839204905 0.5705301110315024
0.002221487979315027 0.004055737713663086 0.5238807345144089 0.7206563610411727


In [90]:
avg_k = []
#Average curvatures
for i in range(0, len(lRfxPairs)):
    avg_k.append([(k1_c[lRfxPairs[i][4]] + k1_c[lRfxPairs[i][5]])/2, (k2_c[lRfxPairs[i][4]] + k2_c[lRfxPairs[i][5]])/2])

lk0, lk1 = [row[0] for row in avg_k], [row[1] for row in avg_k]
#th0 = np.percentile(lk0, 99.5) - np.percentile(lk0, 0.5)
#th1 = np.percentile(lk1, 99.5) - np.percentile(lk1, 0.5)
th0, th1 = max(lk0) - min(lk0), max(lk1) - min(lk1)
rd = round(np.sqrt(th0**2 + th1**2) / 16, 3)

kt_rt = KDTree(avg_k)
z = 158
idx = tk.query_ball_point([(k1_c[lRfxPairs[z][4]] + k1_c[lRfxPairs[z][5]])/2, 
                           (k2_c[lRfxPairs[z][4]] + k2_c[lRfxPairs[z][5]])/2], r = rd, return_sorted=True)
idl = [kx for kx in idx if kx > z]

len(idl) #print(rd, len(idx))

410

In [ ]:
[[row[0], row[1]] for row in avg_k]

In [109]:
avg_k = []
for pair in lRfxPairs:
    avg_k.append([(k1_c[pair[4]] + k1_c[pair[5]])/2, (k2_c[pair[4]] + k2_c[pair[5]])/2])
kt_rt = KDTree(avg_k)

lk0, lk1 = [row[0] for row in avg_k], [row[1] for row in avg_k]
th0 = np.percentile(lk0, 99.5) - np.percentile(lk0, 0.5)
th1 = np.percentile(lk1, 99.5) - np.percentile(lk1, 0.5)
#th0, th1 = max(lk0) - min(lk0), max(lk1) - min(lk1)
rd = round(np.sqrt(th0**2 + th1**2) / 16, 3)
rd

0.038

In [20]:
def BaseContinuousQRotation(pairs):
    lrPairs = []    
    avg_k = []
    #Average curvatures
    for pair in pairs:
        avg_k.append([(k1_c[pair[4]] + k1_c[pair[5]])/2, (k2_c[pair[4]] + k2_c[pair[5]])/2])
    kt_rt = KDTree(avg_k)
    
    lk0, lk1 = [row[0] for row in avg_k], [row[1] for row in avg_k]
    th0 = np.percentile(lk0, 99.5) - np.percentile(lk0, 0.5)
    th1 = np.percentile(lk1, 99.5) - np.percentile(lk1, 0.5)
    #th0, th1 = max(lk0) - min(lk0), max(lk1) - min(lk1)
    rd = round(np.sqrt(th0**2 + th1**2) / 16, 3)
    
    for m in range(0, len(pairs) - 1):
        rpM = pairs[m]
        idx = kt_rt.query_ball_point([(k1_c[rpM[4]] + k1_c[rpM[5]])/2, 
                           (k2_c[rpM[4]] + k2_c[rpM[5]])/2], r = rd, return_sorted=True)
        idl = [kx for kx in idx if kx > m]
                
        for n in idl: #range(m + 1, len(pairs)):
            rpN = pairs[n]
            ratio_k1 = (k1_c[rpM[4]] + k1_c[rpM[5]]) / (k1_c[rpN[4]] + k1_c[rpN[5]])
            ratio_k2 = (k2_c[rpM[4]] + k2_c[rpM[5]]) / (k2_c[rpN[4]] + k2_c[rpN[5]])
            nparallel = abs(np.dot(rpM[2], rpN[2])) < TH_RATIO_K
            
            if TH_RATIO_K < ratio_k1 < 1/TH_RATIO_K and TH_RATIO_K < ratio_k2 < 1/TH_RATIO_K and nparallel:
                #print(m, rpM[2], n, rpN[2])
                line = planesIntersection(rpM, rpN)
                
                p1_i = (v[rpM[0]] + v[rpM[1]]) / 2
                p2_i = (v[rpN[0]] + v[rpN[1]]) / 2
                dist_1i = distancePointLine(p1_i, line[0], line[1]) #distance
                proj_1i = projectPointLine(p1_i, line[0], line[1]) #projection
                
                dist_2i = distancePointLine(p2_i, line[0], line[1]) #distance
                proj_2i = projectPointLine(p2_i, line[0], line[1]) #projection
                
                if np.linalg.norm(proj_1i - proj_2i) < CLUSTER_EP[0] and dist_1i - dist_2i < CLUSTER_EP[0]:
                    #line[0], line[1]
                    rtPair = [rpM[0:2], rpN[0:2], line[0], line[1], rpM[4:6], rpN[4:6], m, n]
                    lrPairs.append(rtPair)
    return lrPairs

In [3]:
start = datetime.now()

lcrPairs = BaseContinuousRotation(lRfxPairs)

end = datetime.now()
print(end - start)

NameError: name 'datetime' is not defined

In [21]:
start = datetime.now()

lcrPairs = BaseContinuousQRotation(lRfxPairs)

end = datetime.now()
print(end - start)

0:00:02.516460


In [26]:
len(lcrPairs)

1243

In [25]:
def distanceRt(rtLineA, rtLineB):    
    distLine = math.sqrt(2 * (1 - abs(np.dot(rtLineA[0], rtLineB[0]))))
    #module = np.linalg.norm(rtLineA) * np.linalg.norm(rtLineB)
    #angle = math.acos(min(1.0, abs(np.dot(rtLineA, rtLineB)) / module))
    
    distProj = np.linalg.norm(rtLineA[1] - rtLineB[1])
    return distProj / diagonal_bbox, distLine

In [23]:
#distances_line / #distance_projections
CLUSTER_RT = [0.075, 0.015]

In [90]:
asigned[2] = False
if not asigned[2]:
    print(1)

1


In [32]:
l_vdir, l_vprj = [], []
for pair in lcrPairs:
    l_vdir.append(pair[2])
    l_vprj.append(pair[3])

kt_vdir = KDTree(l_vdir)
kt_vprj = KDTree(l_vprj)

iq_d = kt_vdir.query_ball_point(l_vdir[0], r = 0.16, return_sorted=True)
iq_p = kt_vprj.query_ball_point(l_vprj[0], r = 30, return_sorted=True)

print(len(iq_d), len(iq_p))

for ix in iq_d:
    print(ix, l_vdir[ix], np.linalg.norm(l_vdir[0] - l_vdir[ix]))
    
for ix in iq_p:
    print(ix, l_vprj[ix], np.linalg.norm(l_vprj[0] - l_vprj[ix]))
    
for el in iq_d:
    if el in iq_p:
        print(el)

23 21
0 [ 0.56090113  0.81147841 -0.16398996] 0.0
135 [ 0.54454479  0.82312121 -0.16106656] 0.020288688019259194
151 [ 0.46366792  0.85667874 -0.22608315] 0.12390695001001738
155 [ 0.46366792  0.85667874 -0.22608315] 0.12390695001001738
230 [ 0.58892042  0.8066679  -0.04959472] 0.1178749000297977
237 [ 0.51499692  0.84232729 -0.15894308] 0.055536663917786375
240 [ 0.51499692  0.84232729 -0.15894308] 0.05553666391778637
302 [ 0.44249596  0.86060729 -0.25209606] 0.15555100717729803
458 [ 0.651308    0.74258245 -0.15610636] 0.11393948658655799
478 [ 0.48987979  0.8176127  -0.30253474] 0.1558085855440575
726 [ 0.51178022  0.80102478 -0.31054839] 0.15492422341192777
728 [ 0.51178022  0.80102478 -0.31054839] 0.1549242234119277
729 [ 0.51178022  0.80102478 -0.31054839] 0.15492422341192766
821 [ 0.66939345  0.73190258 -0.12740102] 0.13943330003621646
846 [ 0.5771079   0.7738428  -0.26098619] 0.105296564484808
917 [ 0.61273367  0.74926707 -0.25130919] 0.11908615083409754
955 [ 0.59144061  0.796

In [159]:
round(diagonal_pbox / 15, 3), round(diagonal_bbox / 15, 3)

(27.443, 14.287)

In [ ]:
lcrPairs[0]

In [27]:
def clusterQRtt(pairs):
    cluster_set = []
    cluster_center = []
    asigned = [False for i in range(len(lcrPairs))]
    
    l_vdir, l_vprj = [], []
    for pair in pairs:
        l_vdir.append(pair[2])
        l_vprj.append(pair[3])
        
    len_xp = max(np.array(l_vprj)[:,0]) - min(np.array(l_vprj)[:,0])
    len_yp = max(np.array(l_vprj)[:,1]) - min(np.array(l_vprj)[:,1])
    len_zp = max(np.array(l_vprj)[:,2]) - min(np.array(l_vprj)[:,2])
    diagonal_pbox = math.sqrt(len_xp * len_xp + len_yp * len_yp + len_zp * len_zp)
    rkd = round(diagonal_pbox / 16, 3)
    
    kt_vdir = KDTree(l_vdir)
    kt_vprj = KDTree(l_vprj)

    for i in range(0, len(pairs)):
        if not asigned[i]:        
            found = False
            c = 0

            for c in range(0, len(cluster_set)):
                distance = distanceRt(pairs[i][2:4], cluster_center[c][0:2])
                if distance[0] < CLUSTER_RT[0] and distance[1] < CLUSTER_RT[1]: #append
                    lcsc = len(cluster_set[c])
                    center = (cluster_center[c][0] * lcsc + pairs[i][2]) / (lcsc + 1)

                    cluster_center[c][0] = center / np.linalg.norm(center)          
                    cluster_center[c][1] = (cluster_center[c][1] * lcsc + pairs[i][3]) / (lcsc + 1)
                    found = True
                    break

            if not found:
                #Get points from query on K-d tree
                iq_d = kt_vdir.query_ball_point(pairs[i][2]
                iq_p = kt_vprj
                idx = kt_rt.query_ball_point([(k1_c[rpM[4]] + k1_c[rpM[5]])/2, 
                    (k2_c[rpM[4]] + k2_c[rpM[5]])/2], r = rd, return_sorted=True)
                idl = [kx for kx in idx if kx > m]
        
                #mode, original points
                cluster = [] #[pairs[i], [], []]
                cluster_set.append(cluster)
                cluster_center.append(pairs[i][2:4])
                c = len(cluster_set) - 1

            cluster_set[c].append(i)
            asigned[i] = True
            #cluster_set[c][1].append(i])
            #cluster_set[c][2].append(pairs[i])
    
    return cluster_set, cluster_center

SyntaxError: invalid syntax (3849737132.py, line 38)

In [81]:
start = datetime.now()

s_RttCluster, c_RttCluster = clusterQRtt(lcrPairs)

end = datetime.now()
print(end - start)

0:00:02.043721


In [82]:
for x in range(0, len(s_RttCluster)):
    if len(s_RttCluster[x]) > 0.01 * len(lcrPairs):
        print(x, len(s_RttCluster[x]), c_RttCluster[x])

7 8 [array([ 0.81403202,  0.57915858, -0.04389993]), array([  82.58847988, -113.15387949,   39.98012416])]
11 10 [array([ 0.05856411,  0.97950138, -0.19273634]), array([ 3.43615315,  1.66972087, 10.73470421])]
13 10 [array([-0.53605014,  0.8361962 , -0.11587132]), array([-102.1655167 ,  -61.33664092,   32.00983859])]
15 8 [array([0.04097265, 0.97324316, 0.2260951 ]), array([  0.24834633, -19.29300044,  81.82262321])]
21 11 [array([ 0.01549046,  0.99986741, -0.00502064]), array([-3.70191163, -0.26636557, 47.64345711])]
26 9 [array([ 0.16681996,  0.98267942, -0.08069857]), array([26.42553602, -2.05786763, 32.82949208])]
27 8 [array([-0.66502321,  0.74680977, -0.0043917 ]), array([-116.02673616, -103.1852514 ,   47.45936966])]
58 9 [array([ 0.08121478,  0.99209316, -0.09568349]), array([ 6.98845345,  2.01572897, 31.83446165])]
62 8 [array([-0.19369663,  0.87598287, -0.44174159]), array([-42.01216509, -26.30832925, -33.14368696])]
64 8 [array([-0.37358851,  0.90241817, -0.21464641]), array

In [72]:
v[443], v[13348], v[376849], v[443]-v[13348], v[443]-v[376849]

(array([ -53.1623 , -159.61   ,    8.37543]),
 array([ -51.219 , -225.841 ,   68.3125]),
 array([-3.61618e+01, -1.77062e+02, -4.41060e-02]),
 array([ -1.9433 ,  66.231  , -59.93707]),
 array([-17.0005  ,  17.452   ,   8.419536]))

In [76]:
for lrPair in lcrPairs:
    if (lrPair[0][0] != lrPair[1][0]) and (lrPair[0][1] != lrPair[1][0]) and (lrPair[0][0] != lrPair[1][1]) and (lrPair[0][1] != lrPair[1][1]):
        print(lrPair)

[[443, 393429], [13348, 253700], array([0.91379906, 0.28645299, 0.28795132]), array([  39.17267918, -178.54336005,   53.30179362]), [1, 607], [26, 401]]
[[35955, 191044], [440895, 489132], array([ 0.55189047,  0.70873723, -0.43944108]), array([102.18564044, -82.70400781,  -5.05216378]), [53, 297], [678, 750]]
[[65983, 197988], [195382, 315883], array([ 0.71695285,  0.45600731, -0.52729114]), array([  79.69462907, -139.19576595,  -12.01801871]), [104, 309], [305, 491]]
[[88043, 150041], [88337, 453895], array([-0.21017624,  0.86086264, -0.46340206]), array([-30.02339113, -16.44980816, -16.94170663]), [133, 224], [134, 697]]
[[88043, 200032], [88337, 453895], array([-0.21068409,  0.85609857, -0.4719189 ]), array([-29.89614069, -15.90900325, -15.51332996]), [133, 313], [134, 697]]
[[88043, 274070], [88337, 238157], array([ 0.09760098,  0.99240015, -0.07493988]), array([ 5.75297012,  2.40754558, 39.37481617]), [133, 430], [134, 380]]


In [ ]:
ctc = 0
for pair in lcrPairs:
    if pair[2][1] > 0.98:
        ctc = ctc + 1
        #print(pair[3])

print(ctc)

In [177]:
len(lcrPairs)

422

In [127]:
diagonal_bbox, CLUSTER_EP

(326.35352314012175, [0.1, 0.1])

In [158]:
lcrPairs[0][0], lcrPairs[0][1]

([781, 3309], [781, 464242])

In [182]:
kdt_vl = KDTree([row[2] for row in lRfxPairs])
kdt_pl = KDTree([row[3] for row in lRfxPairs])

idx = tk.query_ball_point([k1_c[px], k2_c[px]], r = rd, return_sorted=True)

In [56]:
def clusterRtt(pairs):
    cluster_set = []
    cluster_center = []
    
    for i in range(0, len(pairs)):
        found = False
        c = 0
        
        for c in range(0, len(cluster_set)):
            distance = distanceMetricRt(pairs[i][2], pairs[i][3], cluster_center[c][0], cluster_center[c][1])
            if distance[0] < CLUSTER_EP[0] and distance[1] < CLUSTER_EP[1]:                
                center = (cluster_center[c][0] * len(cluster_set[c]) + pairs[i][2]) / (len(cluster_set[c]) + 1)
                cluster_center[c][0] = center / np.linalg.norm(center)
                
                cluster_center[c][1] = (cluster_center[c][1] * len(cluster_set) + pairs[i][3]) / (len(cluster_set) + 1)
                found = True
                break
        
        if not found:
            #mode, original points
            cluster = [] #[pairs[i], [], []]
            cluster_set.append(cluster)
            cluster_center.append(pairs[i][2:4])
            c = len(cluster_set) - 1
        
        cluster_set[c].append(i)
        #cluster_set[c][1].append(i])
        #cluster_set[c][2].append(pairs[i])
    
    return cluster_set, cluster_center

In [59]:
start = datetime.now()

s_RttCluster, c_RttCluster = clusterRtt(lcrPairs)

end = datetime.now()
print(end - start)

0:00:05.349558


In [60]:
for x in range(0, len(s_RttCluster)):
    if len(s_RttCluster[x]) > 0.01 * len(lcrPairs):
        print(x, len(s_RttCluster[x]), c_RttCluster[x])

0 15 [array([0.6828746 , 0.71732545, 0.1382985 ]), array([ 94.99109232, -97.99878138,  56.78738213])]
18 62 [array([-0.03339379,  0.99936719, -0.01225038]), array([-21.24870367,  -2.6765247 ,  51.12680402])]
19 26 [array([-0.14540842,  0.97020914, -0.19377979]), array([-30.35437962,  -2.0448317 ,  10.13724122])]
23 13 [array([-0.29166591,  0.91592732, -0.27569575]), array([-58.4407703 , -14.41657977,   2.55820169])]
43 31 [array([0.09994987, 0.99354013, 0.05374034]), array([ 4.86850711, -1.28399314, 49.09602566])]
45 12 [array([ 0.94327256, -0.29232105,  0.15743338]), array([ -71.74157204, -199.57386784,   37.74565742])]
47 14 [array([0.15991476, 0.97345183, 0.16376446]), array([  9.72384337, -16.29145746,  71.3211235 ])]
51 18 [array([0.25614628, 0.96427114, 0.06760361]), array([ 32.70125963, -16.71781042,  58.2808813 ])]
82 24 [array([ 0.10279721,  0.9547977 , -0.27891589]), array([12.88569988, -3.40939138, -6.81250698])]
86 13 [array([ 0.64069584,  0.76277007, -0.08769644]), array([

In [176]:
for x in range(0, len(s_RttCluster)):
    if len(s_RttCluster[x]) > 0.01 * len(lcrPairs):
        print(x, len(s_RttCluster[x]), c_RttCluster[x])

5 5 [array([-0.33741863,  0.91819369,  0.20753077]), array([-69.41973239, -43.89623493,  88.08157905])]
6 23 [array([-0.12011475,  0.99275655,  0.00262334]), array([-32.86425526,  -4.47872747,  47.58350219])]
10 7 [array([-0.16273117,  0.97218529,  0.16844682]), array([-33.6257338 , -19.81740845,  84.32759415])]
12 12 [array([0.05522142, 0.9576756 , 0.28250318]), array([  1.428726  , -24.89042664,  96.96224476])]
13 7 [array([0.29385907, 0.95026843, 0.10313466]), array([ 52.95850191, -25.0075749 ,  74.648106  ])]
17 5 [array([0.09773538, 0.82775049, 0.5525187 ]), array([  9.26282206, -86.62782739, 131.77332451])]
26 6 [array([0.17105544, 0.31744894, 0.93271979]), array([   2.93452079, -199.89365142,   70.10579612])]
33 7 [array([0.47702814, 0.84741231, 0.23310198]), array([ 72.03328132, -68.64251576,  78.50638451])]
34 6 [array([-0.06241623,  0.76132959, -0.64535376]), array([ -5.22308024, -51.74889733, -58.53539616])]
35 9 [array([ 0.07397857,  0.87894543, -0.47114976]), array([  7.59

In [70]:
for x in range(0, len(s_RttCluster)):
    if len(s_RttCluster[x]) > 0.01 * len(lcrPairs):
        print(x, len(s_RttCluster[x]), c_RttCluster[x])

16 16 [array([-0.22799141,  0.81021875,  0.53996805]), array([-41.50319362, -89.39281161, 122.51649813])]
60 16 [array([-0.22046714,  0.96717381, -0.12636879]), array([-55.47404443, -11.70282231,  20.02116897])]
83 16 [array([-0.01922603,  0.97914556,  0.20224818]), array([-11.23198475, -15.32387706,  81.68518719])]
109 15 [array([ 0.27423143,  0.93668741, -0.21774717]), array([ 50.45747676, -11.04694309,   5.52891717])]
114 19 [array([ 0.23782674,  0.968687  , -0.07130171]), array([40.15069565, -8.03367639, 31.20022319])]
119 38 [array([0.00737507, 0.99976846, 0.02021479]), array([-1.6134617 , -0.15385534, 47.48430304])]
150 14 [array([ 0.03139691,  0.97773633, -0.20747507]), array([-9.9680664 ,  0.47337826,  2.91674295])]


In [73]:
ixc = 0
size = 0
for cl in range(0, len(s_RttCluster)):
    if len(s_RttCluster[cl]) > size:
        ixc = cl
        size = len(s_RttCluster[cl])

ixc, size, c_RttCluster[ixc][0], c_RttCluster[ixc][1]

(119,
 38,
 array([0.00737507, 0.99976846, 0.02021479]),
 array([-1.6134617 , -0.15385534, 47.48430304]))

In [74]:
for ix in s_RttCluster[ixc]:
    print(lcrPairs[ix][0], lcrPairs[ix][1])
    
    #reflectPointLine()

[22478, 200032] [22478, 384440]
[22478, 236269] [22478, 384440]
[23120, 244238] [23120, 336717]
[23120, 244238] [244238, 274070]
[23120, 244238] [244238, 336717]
[23120, 336717] [23120, 507115]
[26271, 131252] [131252, 415971]
[35955, 236269] [35955, 489132]
[37162, 243087] [37162, 422292]
[37162, 243087] [37162, 489132]
[37162, 243087] [243087, 418667]
[37162, 243087] [243087, 422292]
[37576, 418667] [329977, 418667]
[41419, 270760] [82519, 270760]
[46020, 243087] [46020, 422292]
[46020, 243087] [243087, 418667]
[46020, 243087] [243087, 422292]
[46020, 422292] [243087, 422292]
[47594, 329977] [329977, 418667]
[47594, 418667] [243087, 418667]
[47594, 418667] [329977, 418667]
[65983, 392093] [315883, 392093]
[83172, 154768] [154768, 205062]
[83172, 205062] [154768, 205062]
[83761, 182801] [182801, 385792]
[88043, 274070] [244238, 274070]
[132112, 323334] [132112, 507115]
[132112, 423803] [132112, 507115]
[134379, 469211] [263378, 469211]
[150041, 385157] [385157, 492259]
[176168, 486327

In [75]:
v[22478], v[200032],  v[384440], np.linalg.norm(v[22478] - v[200032]), np.linalg.norm(v[22478] - v[384440])

(array([ -36.5159, -178.027 ,   95.7057]),
 array([  47.4219, -180.507 ,   67.2499]),
 array([ -61.3085, -177.321 ,   31.1237]),
 88.66474622125753,
 69.18096685331884)

In [19]:
ix = 7977
jx = 191823

reflexNormal = v[ix] - v[jx]
reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)

normalBalance = n_vertex[ix] + n_vertex[jx]
normalBalance = normalBalance / np.linalg.norm(normalBalance)

if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
        reflexNormal = -reflexNormal

    print(reflexNormal)
    middlePoint = (v[ix] + v[jx]) / 2
    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
    origin = np.array([0, 0, 0])
    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D)

    #Use projection point instead of D-component
    print(ix, jx, reflexNormal, projection)

[ 0.97048185 -0.02651424 -0.23971227]
7977 191823 [ 0.97048185 -0.02651424 -0.23971227] [-9.857313    0.26930869  2.43478932]


In [20]:
ix = 7977
jx = 355387

reflexNormal = v[ix] - v[jx]
reflexNormal = reflexNormal / np.linalg.norm(reflexNormal)

normalBalance = n_vertex[ix] + n_vertex[jx]
normalBalance = normalBalance / np.linalg.norm(normalBalance)

if abs(np.dot(normalBalance, reflexNormal)) < 1 - TH_COSINE:
    if reflexNormal[np.argmax(abs(reflexNormal))] < 0: #Correct direction
        reflexNormal = -reflexNormal

    print(reflexNormal)
    middlePoint = (v[ix] + v[jx]) / 2
    reflexNormal_D = - np.dot(reflexNormal, middlePoint)
    origin = np.array([0, 0, 0])
    projection = projectPointPlane(origin, reflexNormal, reflexNormal_D)

    #Use projection point instead of D-component
    print(ix, jx, reflexNormal, projection)

[0.18601297 0.02885627 0.98212346]
7977 355387 [0.18601297 0.02885627 0.98212346] [ 7.21081964  1.11861754 38.0721575 ]


In [155]:
s_RttCluster, c_RttCluster = clusterRtt(lcrPairs)

In [ ]:
c_RttCluster

In [151]:
type(lcrPairs[0][2:4])

list

In [80]:
n_cluster[0]

[[53,
  75110,
  array([ 0.88089296,  0.24091259, -0.40741714]),
  array([-69.53978822, -19.01821355,  32.16247908])],
 [0],
 [[53,
   75110,
   array([ 0.88089296,  0.24091259, -0.40741714]),
   array([-69.53978822, -19.01821355,  32.16247908])]]]

In [72]:
cluster_set = []
cluster_set.append(lRfxPairs[0][2:4])

In [73]:
cluster_set[0][0] = cluster_set[0][0] * len(cluster_set) + lRfxPairs[1][2]
cluster_set[0][1] = cluster_set[0][1] * len(cluster_set) + lRfxPairs[1][3]
cluster_set

[[array([ 0.77136943,  1.05920065, -0.97169476]),
  array([ -49.46361664, -169.01421519,  135.59718741])]]

In [74]:
np.linalg.norm(cluster_set[0][0])

1.631290141494533

In [69]:
cluster_set[0][0],cluster_set[0][1]

(array([ 0.88089296,  0.24091259, -0.40741714]),
 array([-69.53978822, -19.01821355,  32.16247908]))

In [65]:
lRfxPairs[0][2:4]

[array([ 0.88089296,  0.24091259, -0.40741714]),
 array([-69.53978822, -19.01821355,  32.16247908])]

In [62]:
len(n_cluster)

372

In [164]:
BV_box, BF_box = igl.bounding_box(v)
len(BV_box), len(BF_box), BV_box

(8,
 12,
 array([[  66.1753,    0.    ,  118.817 ],
        [  66.1753,    0.    ,  -16.9141],
        [  66.1753, -259.101 ,  118.817 ],
        [  66.1753, -259.101 ,  -16.9141],
        [ -78.5671,    0.    ,  118.817 ],
        [ -78.5671,    0.    ,  -16.9141],
        [ -78.5671, -259.101 ,  118.817 ],
        [ -78.5671, -259.101 ,  -16.9141]]))

In [170]:
np.mean(v[1:,0]), np.mean(v[1:,1]), np.mean(v[1:,2])

(-8.15843497043787, -199.04268463853882, 49.701276046909214)

In [22]:
np.percentile(v[1:,0], 5), np.percentile(v[1:,0], 20), np.percentile(v[1:,0], 35), np.percentile(v[1:,0], 50), np.percentile(v[1:,0], 65), np.percentile(v[1:,0], 80), np.percentile(v[1:,0], 95)

(-65.2045, -51.3314, -35.8076, -9.24414, 18.3818, 36.445, 47.7787)

In [23]:
np.percentile(v[1:,1], 5), np.percentile(v[1:,1], 20), np.percentile(v[1:,1], 35), np.percentile(v[1:,1], 50), np.percentile(v[1:,1], 65), np.percentile(v[1:,1], 80), np.percentile(v[1:,1], 95)

(-252.051,
 -232.925,
 -213.091,
 -194.674,
 -178.77869999999996,
 -166.385,
 -157.839)

In [21]:
np.percentile(v[1:,1], 5), np.percentile(v[1:,1], 20), np.percentile(v[1:,1], 35), np.percentile(v[1:,1], 50), np.percentile(v[1:,1], 65), np.percentile(v[1:,1], 80), np.percentile(v[1:,1], 95)

(-0.41196209999999944,
 7.03948,
 17.6648,
 31.1237,
 48.4784,
 68.02806,
 82.34387999999994,
 92.0425,
 98.1727)

In [24]:
el = igl.edge_lengths(v, f)
el.shape, type(el)

((866694, 3), numpy.ndarray)

In [25]:
max(el[:][0]), max(el[:][1]), max(el[:][2]), min(el[:][0]), min(el[:][1]), min(el[:][2])

(0.39736015149987347,
 0.36947404617915947,
 0.36947404617915947,
 0.15156384265385478,
 0.15156384265385478,
 0.07661137317657028)

In [26]:
lvx = round(math.ceil(max(max(el[:][0]), max(el[:][1]), max(el[:][2])) * 200)/100, 2)
lvx

0.8

In [27]:
vx = math.ceil(max(v[1:,0]) / lvx) - math.floor(min(v[1:,0]) / lvx) + 1
vy = math.ceil(max(v[1:,1]) / lvx) - math.floor(min(v[1:,1]) / lvx) + 1
vz = math.ceil(max(v[1:,2]) / lvx) - math.floor(min(v[1:,2]) / lvx) + 1

vx, vy, vz

(183, 325, 172)

In [28]:
def locatePoint(point, lv, pdx, pdy, pdz):
    ix = math.ceil(point[0] / lv)
    iy = math.ceil(point[1] / lv)
    iz = math.ceil(point[2] / lv)
    
    return ix - pdx, iy - pdy, iz - pdz

In [29]:
start = datetime.now()

voxelg = np.empty((vx, vy, vz, ), dtype=object)
pdx = math.floor(min(v[1:,0]) / lvx)
pdy = math.floor(min(v[1:,1]) / lvx)
pdz = math.floor(min(v[1:,2]) / lvx)

for vp in range(1, len(v)):
    x, y, z = locatePoint(v[vp], lvx, pdx, pdy, pdz)
    voxelg[x,y,z] = np.append(voxelg[x,y,z], vp)
            
end = datetime.now()
print(end - start)

0:00:07.238686


In [30]:
def findneighbors(ix):
    lx, ly, lz = locatePoint(v[ix], lvx, pdx, pdy, pdz)
    #return np.delete(voxelg[lx,ly,lz], 0), lx, ly, lz
    
    n_voxels = findneighborvoxels(lx, ly, lz)
    nv_list = []
    
    for nx in range(n_voxels.shape[0]):
        for ny in range(n_voxels.shape[1]):
            for nz in range(n_voxels.shape[2]):
                nv_list.extend(np.delete(voxelg[n_voxels[nx][ny][nz][0], n_voxels[nx][ny][nz][1], n_voxels[nx][ny][nz][2]], 0))
    
    return nv_list

In [31]:
x1, y1, z1 = locatePoint(v[12142], lvx, pdx, pdy, pdz)
print(x1, y1, z1, v[12142])

x2, y2, z2 = locatePoint(v[40940], lvx, pdx, pdy, pdz)
print(x2, y2, z2, v[40940])

53 66 133 [ -37.2243 -207.07     88.0148]
54 67 134 [ -36.0692 -205.653    88.8548]


In [32]:
def findneighborvoxels(lx, ly, lz):
    neighbors = np.zeros([3,3,3,3], dtype=int)

    for nx in range(3):
        for ny in range(3):
            for nz in range(3):
                neighbors[nx,ny,nz] = [lx + nx - 1, ly + ny - 1, lz + nz - 1]

    if lx == vx - 1:
        neighbors = np.delete(neighbors, 2, axis = 0)
    if lx == 0:
        neighbors = np.delete(neighbors, 0, axis = 0)

    if ly == vy - 1:
        neighbors = np.delete(neighbors, 2, axis = 1)
    if ly == 0:
        neighbors = np.delete(neighbors, 0, axis = 1)

    if lz == vz - 1:
        neighbors = np.delete(neighbors, 2, axis = 2)
    if lz == 0:
        neighbors = np.delete(neighbors, 0, axis = 2)
    
    return neighbors

In [33]:
ip = 12142
nv = findneighbors(ip)

for nvx in nv:
    if nvx != ip:
        if np.linalg.norm(v[ip] - v[nvx]) < lvx:
            print(np.linalg.norm(v[ip] - v[nvx]), nvx)

0.785706210997468 11805
0.5642921672325477 11809
0.5124028590864881 11813
0.7093535366796989 12133
0.35462827016470233 12136
0.7649322192194545 12255
0.4415997395832699 12256
0.5440334548536628 12259
0.5605798872596043 28
0.5433325685066054 29
0.4247915841915865 11846
0.4021582400001261 11849
0.45341911075735614 11855
0.3545477259833849 30
0.4159405366155017 31
0.5576690775002593 32
0.24047105854967757 12268
0.421997630325106 12275
0.7089248479211308 40764
0.7357440995346142 40855


In [43]:
a_neighbors = findneighborvoxels(54, 66, 133)
a_neighbors[1][0][2], a_neighbors[2][1][0], a_neighbors[0][2][1]

(array([ 54,  65, 134]), array([ 55,  66, 132]), array([ 53,  67, 133]))

In [38]:
voxelg.shape, voxelg[53][66][133][1:]

((183, 325, 172), array([30, 31, 32, 12142, 12268, 12275], dtype=object))

In [44]:
v_indices[0], v1_c[0], v2_c[0], k1_c[0], k2_c[0], np.cross(v1_c[0], v2_c[0]), n_vertex[v_indices[0]]

(53,
 array([-0.18216573,  0.9831671 , -0.01407447]),
 array([-0.98300106, -0.18243106, -0.02068371]),
 0.0010887064928339623,
 0.025401912095101373,
 array([-0.02290316,  0.01006736,  0.999687  ]),
 array([-0.02290316,  0.01006736,  0.999687  ]))

In [48]:
import pandas as pd
k_c = np.abs(k)

In [58]:
vc = 0
vn = 0
for i in range(0, len(adj_lst)):
    if pd.isna(k_c[i]): #if the gaussian curvature is missing
        vc += 1
        #ki_1, ki_2 = calculate_principal(i)
        #print(i, len(adj_lst[i]), bv[i]) #is_border_vertex: True
    
    if k_c[i] == 2 * math.pi: #if the vertex has no neighbors
        #print(i, len(adj_lst[i]), bv[i]) #is_border_vertex: False
        vn += 1

#exceptions: 44645, 44649
print("border vertex:", vc, "isolated", vn)

border vertex: 40 isolated 72215


In [143]:
curvatures = []
for i in range(0, len(adj_lst)):
    if not (pd.isna(k_c[i]) or k_c[i] == 2 * math.pi): #if the vertex has no neighbors
        curvatures.append(k_c[i])

In [159]:
ki_1, ki_2 = calculate_principal(1341)
ki_1, ki_2, ki_1 * ki_2

(0.01121597297798726, 0.03231097757534786, 0.0003623990513774539)

In [82]:
type(v), type(P)

(numpy.ndarray, list)

In [90]:
P = np.array([[21.0, -155, -14]]) # [[0,0,0], [20.0, -154, -13], [19.0, -156, -12]])
S, I, C = igl.signed_distance(P, v, f, return_normals=False)
S, I, C

(array(-0.07538721),
 array(423682, dtype=int32),
 array([  20.93884834, -154.87922546,  -14.06756262]))

In [88]:
v[f[423682][0]], v[f[423682][1]], v[f[423682][2]], v[f[507656][0]], v[f[507656][1]], v[f[507656][2]], v[f[507650][0]], v[f[507650][1]], v[f[507650][2]]

(array([  21.0112, -154.923 ,  -14.2113]),
 array([  20.8611, -154.999 ,  -14.2113]),
 array([  20.8611, -154.733 ,  -13.7358]),
 array([  20.8611, -154.755 ,  -12.7945]),
 array([  20.1527, -154.914 ,  -13.5029]),
 array([  20.2453, -155.006 ,  -12.7946]),
 array([  19.4444, -155.343 ,  -12.7946]),
 array([  20.1527, -155.006 ,  -12.8691]),
 array([  19.9803, -155.006 ,  -13.5029]))

In [82]:
dk = [[k1_c[i] - k2_c[i]] for i in range(len(k1_c))]
max(dk), min(dk)

([-0.0009148522127828112], [-1.0557912553645235])